In [26]:
import os

def delete_pth_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pth'):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# 使用示例
delete_pth_files('results/tests_old')
